In [20]:
import torch
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
# from transformers import RobertaConfig, RobertaModel
# configuration = RobertaConfig()
# model = RobertaModel(configuration)
# configuration = model.config

from transformers import T5Tokenizer, RobertaForMaskedLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")



In [ ]:
# # original text
# text = "ヤクルトの舌や喉に何か残る感じの、あれは何でしょう?すごくいやだ。"

# # prepend [CLS]
# text = "[CLS]" + text

# # tokenize
# tokens = tokenizer.tokenize(text)
# print(tokens)  # output: ['[CLS]', '▁4', '年に', '1', '度', 'オリンピック', 'は', '開かれる', '。']

# # mask a token
# masked_idx = 5
# tokens[masked_idx] = tokenizer.mask_token
# tokens[7] = tokenizer.mask_token
# print(tokens)  # output: ['[CLS]', '▁4', '年に', '1', '度', '[MASK]', 'は', '開かれる', '。']

# # convert to ids
# token_ids = tokenizer.convert_tokens_to_ids(tokens)
# print(token_ids)  # output: [4, 1602, 44, 24, 368, 6, 11, 21583, 8]

# # convert to tensor
# import torch
# token_tensor = torch.LongTensor([token_ids])

# # provide position ids explicitly
# position_ids = list(range(0, token_tensor.size(1)))
# print(position_ids)  # output: [0, 1, 2, 3, 4, 5, 6, 7, 8]
# position_id_tensor = torch.LongTensor([position_ids])

# # get the top 10 predictions of the masked token
# with torch.no_grad():
#     outputs = model(input_ids=token_tensor, position_ids=position_id_tensor)
#     predictions = outputs[0][0, masked_idx].topk(10)

# for i, index_t in enumerate(predictions.indices):
#     index = index_t.item()
#     token = tokenizer.convert_ids_to_tokens([index])[0]
#     print(i, token)


In [ ]:
# original text
text = "ヤクルトの舌や喉に何か残る感じの、あれは何でしょう?すごくいやだ。"

# prepend [CLS]
text = "[CLS]" + text

# tokenize
tokens = tokenizer.tokenize(text)
print(tokens)  # output: ['[CLS]', '▁4', '年に', '1', '度', 'オリンピック', 'は', '開かれる', '。']


# マスク化される文字を一時的に保存する
tmp = ""
for i in range(2, len(tokens)):
    # mask a token
    masked_idx = i
    tmp = tokens[masked_idx]
    tokens[masked_idx] = tokenizer.mask_token
    print(tokens)  # output: ['[CLS]', '▁4', '年に', '1', '度', '[MASK]', 'は', '開かれる', '。']

    # convert to ids
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    print(token_ids)  # output: [4, 1602, 44, 24, 368, 6, 11, 21583, 8]

    # convert to tensor
    import torch
    token_tensor = torch.LongTensor([token_ids])

    # provide position ids explicitly
    position_ids = list(range(0, token_tensor.size(1)))
    print(position_ids)  # output: [0, 1, 2, 3, 4, 5, 6, 7, 8]
    position_id_tensor = torch.LongTensor([position_ids])

    # get the top 10 predictions of the masked token
    with torch.no_grad():
        outputs = model(input_ids=token_tensor, position_ids=position_id_tensor)
        predictions = outputs[0][0, masked_idx].topk(10)

    for i, index_t in enumerate(predictions.indices):
        index = index_t.item()
        token = tokenizer.convert_ids_to_tokens([index])[0]
        print(i, token)
    
    # mask　から 通常の文字に戻す
    tokens[masked_idx] = tmp


# 暗示的怒り

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

print(os.environ['CUDA_LAUNCH_BLOCKING'])



1


In [2]:
import torch
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
import pandas as pd
dataset = pd.read_json("ksl_pub/suggestive_anger/data/fuman_survey/json/human_data_ver5.json")
print(f'データサイズ： {dataset.shape}')

# for i in range(len(dataset.fulltext.values)):
#     print(dataset.label.values[i],  dataset.fulltext.values[i])

# データの抽出
sentences = dataset.fulltext.values
labels = dataset.label.values

dataset.sample(3)

データサイズ： (1800, 5)


,fulltext,sentences,danwa_result,result_form,label
14,チェキカメラ、内カメラでもとれるようにしてほしい,[チェキカメラ、内カメラでもとれるようにしてほしい],短文,"{'4': '1.0', '5': '2.0', '2': '6.0', '1': '3.0...",2
1721,カチャカチャと音がするものしかない。無音のものがあるといいのに。,"[カチャカチャと音がするものしかない, 無音のものがあるといいのに]",要望,"{'5': '1.0', '4': '1.0', '2': '1.0', '1': '5.0...",1
19,最近の雑誌は高い中身の内容薄いのに千円近くするおまけもいらない,[最近の雑誌は高い中身の内容薄いのに千円近くするおまけもいらない],順接,"{'4': '2.0', '0': '6.0', '1': '3.0', '2': '2.0'}",0


In [4]:
# 1. BERT Tokenizerを用いて単語分割・IDへ変換
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
# from transformers import BertJapaneseTokenizer
# tokenizer = BertJapaneseTokenizer.from_pretrained('rinna/japanese-roberta-base')

2023-02-10 17:55:50.847849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 17:55:52.951002: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-10 17:55:52.951096: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-10 17:55:52.951106: W tensorfl

In [5]:
## テスト実行
# 元文章
for i in range(3):
    print('Original: ', sentences[i])
    print('Tokenized: ', tokenizer.tokenize(sentences[i]))    # Tokenizer
    print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[i])))    # Token-id
    print("\n\n")

Original:  最近の内容は、のび太が、他力本願ですぐ調子にのり、エゴが強すぎると感じている。子供に見せたくないアニメになった。
Tokenized:  ['最近', 'の', '内容', 'は', '、', 'のび太', 'が', '、', '他', '##力', '##本', '##願', 'で', 'すぐ', '調子', 'に', 'のり', '、', 'エ', '##ゴ', 'が', '強', 'すぎる', 'と', '感じ', 'て', 'いる', '。', '子供', 'に', '見せ', 'たく', 'ない', 'アニメ', 'に', 'なっ', 'た', '。']
Token IDs:  [5233, 5, 1386, 9, 6, 15217, 14, 6, 375, 28677, 28515, 29645, 12, 2459, 12327, 7, 6317, 6, 133, 28796, 14, 427, 8896, 13, 3415, 16, 33, 8, 1803, 7, 2685, 5034, 80, 1149, 7, 58, 10, 8]



Original:  ヤクルトの舌や喉に何か残る感じの、あれは何でしょう?すごくいやだ。
Tokenized:  ['ヤクルト', 'の', '舌', 'や', '喉', 'に', '何', 'か', '残る', '感じ', 'の', '、', 'あれ', 'は', '何', 'でしょ', 'う', '?', 'すご', '##く', 'いや', 'だ', '。']
Token IDs:  [11489, 5, 10694, 49, 14355, 7, 1037, 29, 4105, 3415, 5, 6, 2787, 9, 1037, 11655, 205, 2935, 14993, 28504, 17989, 75, 8]



Original:  めばちこ痛い。治ってもまたできる...
Tokenized:  ['め', '##ば', '##ち', 'こ', '痛', '##い', '。', '治', '##っ', 'て', 'も', 'また', 'できる', '...']
Token IDs:  [2087, 28620, 28586, 27, 4897, 28457, 8, 131

In [6]:
# 最大単語数の確認
max_len = []
# 1文づつ処理
for sent in sentences:
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))
# 最大の値を確認
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値(' + str(max(max_len) + 2) + ')が最大単語数')

最大単語数:  185
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値(187)が最大単語数


In [7]:
import torch

input_ids = []
attention_masks = []

# 1文づつ処理
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        # max_length = 187,           # 文章の長さを固定（Padding/Trancatinating）
                        max_length = max(max_len) + 2,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels, device=device)

# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/nb-user/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  最近の内容は、のび太が、他力本願ですぐ調子にのり、エゴが強すぎると感じている。子供に見せたくないアニメになった。
Token IDs: tensor([    2,  5233,     5,  1386,     9,     6, 15217,    14,     6,   375,
        28677, 28515, 29645,    12,  2459, 12327,     7,  6317,     6,   133,
        28796,    14,   427,  8896,    13,  3415,    16,    33,     8,  1803,
            7,  2685,  5034,    80,  1149,     7,    58,    10,     8,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,

In [8]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90%地点のIDを取得
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('訓練データ数：{}'.format(train_size))
print('検証データ数:　{} '.format(val_size))

# データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

訓練データ数：1620
検証データ数:　180 


In [9]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 6, # ラベル数
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
).to(device)

# モデルをGPUへ転送
# 下の方でエラーが発生するため、一時的に無効化
model.cuda()


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [10]:
# # 最適化手法の設定
# optimizer = AdamW(model.parameters(), lr=2e-5)

# # 訓練パートの定義
# def train(model):
#     model.train() # 訓練モードで実行
#     train_loss = 0
#     for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         b_labels = batch[2].to(device)
#         optimizer.zero_grad()
#         loss, logits = model(b_input_ids, 
#                              token_type_ids=None, 
#                              attention_mask=b_input_mask, 
#                              labels=b_labels)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         optimizer.step()
#         train_loss += loss.item()
#     return train_loss

# # テストパートの定義
# def validation(model):
#     model.eval()# 訓練モードをオフ
#     val_loss = 0
#     with torch.no_grad(): # 勾配を計算しない
#         for batch in validation_dataloader:
#             b_input_ids = batch[0].to(device)
#             b_input_mask = batch[1].to(device)
#             b_labels = batch[2].to(device)
#             with torch.no_grad():        
#                 (loss, logits) = model(b_input_ids, 
#                                     token_type_ids=None, 
#                                     attention_mask=b_input_mask,
#                                     labels=b_labels)
#             val_loss += loss.item()
#     return val_loss

In [11]:
# 最適化手法の設定
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader: # train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        loss = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels).loss # 戻り値とここを修正

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval() # 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                loss = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels).loss # 戻り値とここを修正
            val_loss += loss.item()
    return val_loss

In [12]:
# 学習の実行
max_epoch = 1
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    test_ = train(model)
    train_loss_.append(train_)
    test_loss_.append(test_)
train_loss_

[74.00467312335968]

In [13]:
# 検証方法の確認（1バッチ分で計算ロジックに確認）

model.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

In [14]:
## 予測結果の確認
print(f'出力:{preds}')

出力:SequenceClassifierOutput(loss=None, logits=tensor([[-0.1078,  2.3627, -1.7634, -0.3342, -0.5414, -0.6090],
        [ 0.6518,  0.3105, -1.8580,  0.0424, -0.5819,  0.5271],
        [-1.3868,  0.8976, -0.2736, -0.9676,  0.3157, -0.4518],
        [-0.2521,  2.3922, -1.6009, -0.3352, -0.3784, -0.8180],
        [ 0.6905,  0.6604, -1.8571,  0.1842, -0.5850,  0.2643],
        [ 0.6086,  1.1970, -1.9666,  0.2025, -0.6530, -0.0635],
        [ 0.5671,  1.1594, -1.4935, -0.3725, -0.6269,  0.1761],
        [-1.8114,  0.0439,  3.3915, -0.2929, -1.1834, -1.1395],
        [-0.5611,  0.5423, -1.7004, -0.2798,  0.2394,  0.0158],
        [-1.4774,  0.6904, -0.4987, -0.7212,  0.5240, -0.3153],
        [-1.6062,  0.4561,  3.7199, -0.8118, -1.1322, -1.0862],
        [-1.5611,  0.0809,  3.0536, -0.1876, -0.9983, -1.2292],
        [-0.9971,  1.2709, -0.3272, -0.2107,  0.1839, -0.8773],
        [-1.5705, -0.0751,  3.1134, -0.2249, -1.0056, -1.1878],
        [-1.4181,  1.9073,  0.8834, -0.8319, -0.8379, -0.6

In [16]:
# 比較しやすい様にpd.dataframeへ整形
import pandas as pd
import numpy as np
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['感情的攻撃', '感情的説得', '理性的説得', '嫌味・皮肉', '遠回し', '怒ってない'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

,感情的攻撃,感情的説得,理性的説得,嫌味・皮肉,遠回し,怒ってない,pred_label,true_label
0,-0.107773,2.362728,-1.763384,-0.334184,-0.541413,-0.608966,1,1
1,0.651800,0.310498,-1.857996,0.042417,-0.581905,0.527077,0,3
2,-1.386837,0.897578,-0.273604,-0.967604,0.315678,-0.451812,1,1
3,-0.252058,2.392193,-1.600874,-0.335171,-0.378388,-0.818017,1,1
4,0.690539,0.660420,-1.857126,0.184235,-0.585034,0.264289,0,0


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name())
# > Tesla T4
print(torch.cuda.get_device_capability())
# > (7, 5)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# 変数を用意
a = torch.randn(5)
b = torch.randn(5)

# 今回は変数aだけをGPUに送る。
a = a.to(device)
b = b.to(device)

print(a + b)